# Builder (строитель)

Строитель — это порождающий паттерн проектирования, который позволяет создавать сложные объекты пошагово. Строитель даёт возможность использовать один и тот же
код строительства для получения разных представлений объектов.


## Пример

Автоматизируем процесс продажи "сборок", т. е. уже собранных компьютеров. Для сборки компьютера, нужно установить комплектующие в правильном порядке. Комплектующих может быть много.


### Проблема

**Наивная реализация.** Создаем класс `Computer` и заполняем значения через атрибуты. В комментариях описаны возникающие проблемы.


In [9]:
class Computer:
    cpu: str
    memory: str
    hard_drive: str
    video_card: str


# Данному случае ломается инкапсуляция
# плюс, мы не можем гарантировать порядок присваивания в других местах, если он нам важен
computerA = Computer()
computerA.cpu = "Intel Core i7"
computerA.memory = "16 GB"
computerA.hard_drive = "SSD"
computerA.video_card = "Nvidia"

Пробуем задать параметры через конструктор.


In [12]:
from dataclasses import dataclass

# Сгенерируем __init__ через `dataclass`
ComputerWithConstructor = dataclass(Computer)

# Проблема заключается в том, что параметров может быть очень много
ComputerB = ComputerWithConstructor(
    cpu="Intel Core i7",
    memory="16 GB",
    hard_drive="SSD",
    video_card="Nvidia",
)

### Решение

Используем отдельные классы, реализующие интерфейс `AbsBuilder` для комплектования конкретных "сборок". Дополнительно выделяем класс `Director`, который будет отвечать за правильный порядок сборки компьютеров.


In [18]:
import abc


class Computer:
    cpu: str
    memory: str
    hard_drive: str
    video_card: str

    def display(self) -> None:
        print("CPU:        ", self.cpu)
        print("Memory:     ", self.memory)
        print("HD:         ", self.hard_drive)
        print("Video card: ", self.video_card)


class AbsBuilder(abc.ABC):
    def get_computer(self) -> Computer:
        """
        Возвращает текущий собранный компьютер
        """
        return self._computer

    def new_computer(self) -> None:
        """
        Создает заготовку для сборки компьютера
        """
        self._computer = Computer()

    @abc.abstractmethod
    def install_mainboard(self) -> None:
        ...

    @abc.abstractmethod
    def install_hard_drive(self) -> None:
        ...

    @abc.abstractmethod
    def install_video_card(self) -> None:
        ...


class IntelComputerBuilder(AbsBuilder):
    """
    Конкретный сборщик
    """

    def install_mainboard(self) -> None:
        self._computer.cpu = "Intel Core i7"
        self._computer.memory = "16 GB"

    def install_hard_drive(self) -> None:
        self._computer.hard_drive = "SSD"

    def install_video_card(self) -> None:
        self._computer.video_card = "Nvidia"


class AMDComputerBuilder(AbsBuilder):
    """
    Конкретный сборщик
    """

    def install_mainboard(self) -> None:
        self._computer.cpu = "AMD Ryzen 7"
        self._computer.memory = "32 GB"

    def install_hard_drive(self) -> None:
        self._computer.hard_drive = "HDD"

    def install_video_card(self) -> None:
        self._computer.video_card = "AMD"


class Director:
    """
    Класс знает как правильно собирать компьютер
    т. е. владеет знанием о порядке вызовов методов у сборщиков
    """

    def __init__(self, builder: AbsBuilder) -> None:
        self._builder = builder

    def build_computer(self) -> None:
        self._builder.new_computer()
        self._builder.install_mainboard()
        self._builder.install_hard_drive()
        self._builder.install_video_card()

    def get_computer(self) -> Computer:
        return self._builder.get_computer()

In [23]:
# Собираем конфигурацию на Intel
intel_builder = Director(IntelComputerBuilder())
intel_builder.build_computer()
computer = intel_builder.get_computer()
computer.display()

print()

# Собираем конфигурацию на AMD
amd_builder = Director(AMDComputerBuilder())
amd_builder.build_computer()
computer = amd_builder.get_computer()
computer.display()

CPU:         Intel Core i7
Memory:      16 GB
HD:          SSD
Video card:  Nvidia

CPU:         AMD Ryzen 7
Memory:      32 GB
HD:          HDD
Video card:  AMD
